In [43]:
! pip install datasets classy-fire scikit-learn openai


[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
from datasets import load_dataset
from classy_fire import LLMClassifier
from dotenv import load_dotenv
from openai import InvalidRequestError
import random

# requires a .env file with OPENAI_API_BASE, OPENAI_API_VERSION, OPENAI_API_TYPE and OPENAI_API_KEY variables in the same directory as this notebook
load_dotenv()

# Loading an arbitrary text classification dataset from huggingface
dataset = load_dataset("tweet_eval", "emoji")

ds = dataset['test']
classifier = LLMClassifier(ds.features['label'].names
                           task_description="The input is a tweet and the options are emojis that would best match the tweet.",
                          )
print(ds.features['label'].names)

SyntaxError: invalid syntax (2586473530.py, line 15)

In [59]:
classifier('That is the funniest thing I have ever heard!')

('😂', 2)

In [60]:
classifier('I love you!')

('💕', 3)

In [61]:
y_true, y_pred = [], []
random.seed(42)
for smpl in random.choices(ds, k=100):
    txt, y = smpl['text'], smpl['label']
    try:
        yhat = classifier(txt)[1]
    except InvalidRequestError:
        print(f'The following input failed: {txt}')
        continue
    y_true.append(y)
    y_pred.append(yhat)

The following input failed: Wildwood 2k17 w my girl ️ @ Wildwood, New Jersey
The following input failed: These girls. ️ ️ ️#girlfriends #collegefriends @ Park Hyatt Aviara…


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


In [62]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred, target_names=ds.features['label'].names, labels=list(set(y_pred))))

              precision    recall  f1-score   support

           ❤       1.00      0.30      0.46        10
           😍       0.08      0.50      0.13         2
           😂       0.36      1.00      0.53         4
           💕       0.29      1.00      0.44         2
           🔥       0.00      0.00      0.00        11
           😊       0.00      0.00      0.00         5
           😎       0.00      0.00      0.00         2
           ✨       0.07      0.60      0.12         5
           💙       0.00      0.00      0.00         3
           😘       1.00      0.33      0.50         3
           📷       1.00      1.00      1.00         3
          🇺🇸       0.12      0.14      0.13         7

   micro avg       0.18      0.32      0.23        57
   macro avg       0.33      0.41      0.28        57
weighted avg       0.34      0.32      0.24        57



c:\python39\env\lib\site-packages\sklearn\metrics\_classification.py:2141: UserWarning: labels size, 12, does not match size of target_names, 20
  warnings.warn(
